In [6]:
# for getting environment variables for API keys
import os
import requests
# for scraping data and gettting encodings
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
import content_extraction
# SQL stuff
import psycopg2
from psycopg2 import extensions
from pgvector.psycopg2 import register_vector
import psycopg2.extras
# gemini 
from google import genai
from google.genai import types

In [7]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY_2")
NEWS_API_KEY = os.getenv("NEWS_API_KEY")
POSTGRESQL_PWD = os.getenv("POSTGRESQL_PWD")

In [8]:
# ONLY RUN WHEN NEEDED
url = ('https://newsapi.org/v2/everything?'
       'language=en&'
       'q=Gaza&'
       'from=2025-07-15&'
       'sortBy=relevancy&'
       'pageSize=100&'
       f'apiKey={NEWS_API_KEY}')

response = requests.get(url)
print(response.json())

{'status': 'ok', 'totalResults': 11196, 'articles': [{'source': {'id': 'wired', 'name': 'Wired'}, 'author': 'David Cox', 'title': 'The Grave Long-Term Effects of the Gaza Malnutrition Crisis', 'description': 'The WHO believes one in five children in Gaza are acutely malnourished, with the health effects potentially lasting for generations.', 'url': 'https://www.wired.com/story/how-malnutrition-affects-the-body-gaza-hunger-children-health/', 'urlToImage': 'https://media.wired.com/photos/688901c7cf31e8c7b9d4237f/191:100/w_1280,c_limit/Malnutrition-Gaza-Science-2226867552.jpg', 'publishedAt': '2025-07-31T17:22:53Z', 'content': 'The moment Merry Fitzpatrick realized that Gazas malnutrition crisis had progressed to a newer and deadlier phase was when surgeons at the few hospitals still operational on the Strip reported that w… [+3282 chars]'}, {'source': {'id': None, 'name': 'BBC News'}, 'author': None, 'title': 'Taking control means abandoning hostages, families say', 'description': "Prime

In [9]:
#---------------------- HELPERS ----------------------# 
def ask_gemini(prompt, content, instruction, test_mode=False):
    if test_mode:
        return "In test mode, will not call API"
    client = genai.Client(api_key=GEMINI_API_KEY)

    contents = prompt + content

    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        config=types.GenerateContentConfig(
            system_instruction=instruction
        ),
        contents=contents
    )
    
    return response.text

def call_news_api(keyword, date, NEWS_API_KEY, everything=True):
    API_KEY = NEWS_API_KEY
    
    # choose endpoint
    endpoint = "everything" if everything else "top-headlines"
    # build URL with the passed-in variables
    url = (
        f"https://newsapi.org/v2/{endpoint}"
        f"?q={keyword}"
        f"&from={date}"
        "&sortBy=relevancy"
        f"&apiKey={API_KEY}"
    )
    response = requests.get(url)

    return response

def get_keyword(query):
    """
    simply return the main keyword of the query the user has asked
    """
    keyword_model = KeyBERT('distilbert-base-nli-mean-tokens')
    keywords = keyword_model.extract_keywords(query)            # a list of keywords with the most relevant listed first

    return keywords[0][0]

def add_embeddings(article_chunks, sentence_model):
    """
    input: list of article chunks
    output: chunks with embeddings
    """
    
    for chunk in article_chunks:
        
        embedding = sentence_model.encode(chunk.content)

        chunk.embed(embedding)


#------------------------- SQL STUFF -------------------------#
def store_sql(article_chunks):
    """
    stores chunks into a postgreSQL database. 
    input: list of objects of class ArticleChunk, that have vector embeddings
    """
    # SQL connection params
    hostname = 'localhost'
    database = 'newsdash'
    username = 'postgres'
    pwd = POSTGRESQL_PWD
    port_id = 5432
    conn = None
    cur = None

    print("article chunk length: ", len(article_chunks))
    
    try:
        # initializing connection object
        conn = psycopg2.connect(
                    host = hostname,
                    database = database,
                    user = username,
                    password = pwd,
                    port = port_id)

        register_vector(conn)

        # Turn on autocommit so CREATE EXTENSION runs immediately
        conn.set_isolation_level(extensions.ISOLATION_LEVEL_AUTOCOMMIT)
        
        # open a cursor, performs SQL operations
        cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        cur.execute('CREATE EXTENSION IF NOT EXISTS "uuid-ossp";')
        cur.execute("DROP TABLE IF EXISTS chunked_data;")

        # Switch back to transactional mode to make the table
        conn.set_isolation_level(extensions.ISOLATION_LEVEL_READ_COMMITTED)
        
        create_script = ''' CREATE TABLE IF NOT EXISTS chunked_data (
                                chunk_id UUID NOT NULL PRIMARY KEY,
                                author VARCHAR(50),
                                url VARCHAR(300),
                                title VARCHAR(200),
                                source VARCHAR(100),
                                content TEXT,
                                embedding VECTOR(384))'''
        
        cur.execute(create_script)
        # insert data into the table

        insert_script = 'INSERT INTO chunked_data (chunk_id, author, url, title, source, content, embedding) VALUES (uuid_generate_v4(), %s, %s, %s, %s, %s, %s)'
        
        for chunk in article_chunks:
            data = [chunk.author, chunk.url, chunk.title, chunk.source, chunk.content, chunk.embedding]
            cur.execute(insert_script, data)

        # always place this at the bottom
        conn.commit()

    except Exception as error:
        import traceback; traceback.print_exc()
        raise
    finally:
        # close the cursor and the connection
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

def fetch_sql(encoded_query, n_chunks):
    """
    stores chunks into a postgreSQL database. 
    input: list of objects of class ArticleChunk, that have vector embeddings
    """
    # SQL connection params
    hostname = 'localhost'
    database = 'newsdash'
    username = 'postgres'
    pwd = POSTGRESQL_PWD
    port_id = 5432
    conn = None
    cur = None
    
    try:
        # initializing connection object
        conn = psycopg2.connect(
                    host = hostname,
                    database = database,
                    user = username,
                    password = pwd,
                    port = port_id)

        register_vector(conn)

        # Turn on autocommit so CREATE EXTENSION runs immediately
        conn.set_isolation_level(extensions.ISOLATION_LEVEL_AUTOCOMMIT)
        
        # open a cursor, performs SQL operations
        cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

        # Switch back to transactional mode to make the table
        conn.set_isolation_level(extensions.ISOLATION_LEVEL_READ_COMMITTED)

        # fetching data
        featch_script ="""
        SELECT
            chunk_id,
            author,
            url,
            title,
            source,
            content,
            embedding <=> %s   AS cosine_dist
        FROM chunked_data
        ORDER BY cosine_dist
        LIMIT %s;
        """

        cur.execute(featch_script, (encoded_query, n_chunks))
        top_n_chunks = cur.fetchall()
        
        print("SQL select completed")
        # always place this at the bottom
        conn.commit()

    except Exception as error:
        import traceback; traceback.print_exc()
        raise
    finally:
        # close the cursor and the connection
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()
        
        return top_n_chunks
        

#------------------------- GEMINI -------------------------#
def read_with_gemini(top_n_chunks):
    """
    feed the chunks, a prompt, and system instructions into gemini
    """
    just_contnent = []

    for chunk in top_n_chunks:
        just_contnent.append(chunk['content'])

    # turn the list of chunks into a string
    sep = ".\n"
    content = sep.join(just_contnent)
    
    instructions = "You don't know anything except the information provided for you. Base your answer solely off of this information provided."
    prompt = "Evaluate the validity of the users question, or generate an accurate summary from the information provided."

    gemini_response = ask_gemini(prompt, content, instructions, test_mode=True)

    return gemini_response

def get_citation(articles):
    """
    get citation format of the sources used
    """
    final_citation = "Response is based on the following sources: \n"
    citation_list = []

    for article in articles:
        citation_list.append(
                        f"{article['author']}. "
                        f"\"{article['title']}\". "
                        f"{article['source']}. "
                        f"{article['url']}\n"
                    )
        
    # remove duplicates
    unique_citation = list(set(citation_list))

    # turn into string
    sep = " "
    final_citation += sep.join(unique_citation)
    final_citation = final_citation.replace("None.", "").strip()
    
    return final_citation


In [10]:
def new_rag_system():
    """
    1. get a user input, use its keyword to search the API
    2. rewrite the said content of each in a list of articles, each is a dict
    3. chunk up the data, with some overlap between chunks, also get an embedding for each of them
    4. store each of these chunks, along with info into a database
    5. gather the N most similar chunks to the user's query 
    6. feed these chunks into the LLM prompt
    7. return the answer as well as the sources used
    """

    # 1. get a user input, use its keyword to search the API
    # user input
    query = "Gaza conflict"
    date = "2025-07-01"
    everything = True
    sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
    n_chunks = 5             # how many chunks to base llm response on
    n_articles = 20          # how many articles to base llm response on

    # put this keyword into the API search as its "q" value and sort by relevance
    keyword = get_keyword(query)

    # call the news_api with a set of parameters, remember there's a limit so comment this out whenever you can.
    # response = call_news_api(keyword, date, NEWS_API_KEY, everything)  
    response_data = response.json()

    # 2. rewrite the said content of each in a list of articles, each is a dict
    # format the articles into a list called all_articles. each item in this list is a dictionary
    all_articles = response_data.get('articles', [])[:n_articles] 

    # write the actual content into this list of articles
    content_extraction.extract_content(all_articles)
    all_articles = [a for a in all_articles if a['content'] is not None]

    # 3. chunk up the data, with some overlap between chunks and add embeddings
    article_chunks = content_extraction.chunkify(all_articles)
    add_embeddings(article_chunks, sentence_model)

    # 4. store each of these chunks, along with info into a postgreSQL database
    store_sql(article_chunks)

    # 5. gather the N most similar chunks to the user's query 
    encoded_query = sentence_model.encode(query)
    top_n_chunks = fetch_sql(encoded_query, n_chunks)

    # 6. feed these chunks into the LLM prompt
    final_answer = read_with_gemini(top_n_chunks)
    
    # 7. return the answer as well as the sources used
    citation = get_citation(top_n_chunks)

    print(final_answer)
    print(citation)
    
new_rag_system()

article chunk length:  52
SQL select completed
In test mode, will not call API
Response is based on the following sources: 
 "Jeremy Bowen: Israel's allies see evidence of war crimes in Gaza mounting up". BBC News. https://www.bbc.com/news/articles/cp863mln0pmo
  "Jeremy Bowen: Israel's aid measures a gesture to allies horrified by Gaza starvation". BBC News. https://www.bbc.com/news/articles/cz60x5v75p1o
  "What Israel's Gaza City takeover plan could mean for Palestinians". BBC News. https://www.bbc.com/news/videos/cj4wexw5gwyo
  "US diplomat says UK would have lost WW2 with Starmer as leader". BBC News. https://www.bbc.com/news/articles/c5yp78gd636o
